In [169]:
import torch
from torch import nn
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor

In [170]:
training = datasets.MNIST("data", train=True,  download=True, transform=ToTensor())
test = datasets.MNIST("data", train=False,  download=True, transform=ToTensor())

In [171]:
train_dataloader = DataLoader(training, batch_size=64)
test_dataloader = DataLoader(test, batch_size=64)

In [172]:
cnn_train_dataloader = DataLoader(training, batch_size=64)
cnn_test_dataloader = DataLoader(test, batch_size=64)

In [173]:
for images, label in cnn_train_dataloader:
    images = images.view(-1, 1, 28, 28)
for images, label in cnn_test_dataloader:
    images = images.view(-1, 1, 28, 28)

In [174]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [175]:
class NeuralNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 300, device=device),
            nn.BatchNorm1d(300),
            nn.ReLU(),
            nn.Linear(300, 100),
            nn.BatchNorm1d(100),
            nn.ReLU(),
            nn.Linear(100, 10)
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [176]:
class CNN(nn.Module):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.conv_net = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.fc_net = nn.Sequential(
            nn.Linear(7*7*64, 300),
            nn.BatchNorm1d(300),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(300, 100),
            nn.BatchNorm1d(100),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(100, 10)
        )

    def forward(self, x):
        x = self.conv_net(x)
        x = x.view(x.size(0), -1)
        x = self.fc_net(x)
        return x

In [177]:
model = NeuralNet()
model.load_state_dict(torch.load("MNIST_weights.pth"))

<All keys matched successfully>

In [178]:
model_cnn = CNN()
model_cnn.load_state_dict(torch.load("MNIST_cnn_weights.pth"))

<All keys matched successfully>

In [179]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
optimizer_adam = torch.optim.Adam(model.parameters(), lr=1.8e-3)

In [185]:
loss_fn_cnn = nn.CrossEntropyLoss()
optimizer_cnn = torch.optim.Adam(model_cnn.parameters(), lr=2.5e-3)

In [181]:
def train_loop(dataloader, model, loss_fn, optimizer):
    model.train()

    for batch, (X, Y) in enumerate(dataloader):
        pred = model(X)
        loss = loss_fn(pred, Y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            print(f"Loss: {loss.item()}, batch: {batch}")

In [182]:
def test_loop(dataloader, model, loss_fn, iteration):
    num_batches = len(dataloader)
    size = len(dataloader.dataset)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for (X, Y) in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, Y)
            correct += (pred.argmax(1) == Y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size

    print(f"Test {iteration+1} Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


In [183]:
epochs = 2
for t in range(epochs):
    train_loop(train_dataloader, model, loss_fn, optimizer_adam)
    test_loop(test_dataloader, model, loss_fn, t)
torch.save(model.state_dict(), "MNIST_weights.pth")

In [184]:
epochs2 = 1
for t in range(epochs2):
    train_loop(cnn_train_dataloader, model_cnn, loss_fn_cnn, optimizer_cnn)
    test_loop(cnn_test_dataloader, model_cnn, loss_fn_cnn, t)
print("DONE")
torch.save(model_cnn.state_dict(), "MNIST_cnn_weights.pth")

Loss: 0.00823370274156332, batch: 0
Loss: 0.0548979751765728, batch: 100
Loss: 0.019660314545035362, batch: 200
Loss: 0.04747716709971428, batch: 300
Loss: 0.06134342402219772, batch: 400
Loss: 0.09203587472438812, batch: 500
Loss: 0.038017965853214264, batch: 600
Loss: 0.09057608246803284, batch: 700
Loss: 0.1557389199733734, batch: 800
Loss: 0.09154873341321945, batch: 900
Test 1 Error: 
 Accuracy: 98.5%, Avg loss: 0.054713 

Loss: 0.010318008251488209, batch: 0
Loss: 0.058080460876226425, batch: 100
Loss: 0.01949397474527359, batch: 200
Loss: 0.011125986464321613, batch: 300
Loss: 0.015666283667087555, batch: 400
Loss: 0.09918995946645737, batch: 500
Loss: 0.030022313818335533, batch: 600
Loss: 0.025334451347589493, batch: 700
Loss: 0.1753470003604889, batch: 800
Loss: 0.03216719627380371, batch: 900
Test 2 Error: 
 Accuracy: 98.5%, Avg loss: 0.053555 

Loss: 0.0060417805798351765, batch: 0
Loss: 0.009454065002501011, batch: 100
Loss: 0.03304034844040871, batch: 200
Loss: 0.02707076